In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from scipy import stats
from scipy.stats import norm
%matplotlib inline
import datetime
import math

In [85]:
EyeQ = pd.read_csv('cantsay.csv')
Spire = pd.read_csv('wonttell.csv')

In [4]:
# EyeQ = pd.read_csv('SumEye.csv')

# adjusting the time to sync

In [54]:
Spire['time'] = pd.to_datetime(Spire['time'])

In [1]:
# Spire.head()

basic time fix, just for sake of getting data to work with, better version in Tian's algorithym

In [6]:
# Spire['synchtime'] = Spire['time'] - datetime.timedelta(hours = 1, seconds = 10)

In [56]:
Spire.loc[Spire['RCPE_NM'].str.contains('LMN'), 'LMN'] = 1
Spire.loc[Spire['RCPE_NM'].str.contains('CHR'), 'CHR'] = 1
Spire.loc[Spire['RCPE_NM'].str.contains('STW'), 'STW'] = 1
Spire.loc[Spire['RCPE_NM'].str.contains('VAN'), 'VAN'] = 1

In [57]:
Spire['beverage']=Spire['RCPE_NM'].str.split('|', expand = True)[0]

In [3]:
# Spire.head()

changing the spire data to aggregate quantities over event, change recipe name to be columns

In [59]:
sum_Spire = pd.pivot_table(Spire, index='event', columns= 'beverage', 
                        values='INVEN_TX_CMPNT_QTY', aggfunc=np.sum)

getting the spire datetimes

In [2]:
sum_Spire2 = Spire.groupby('event').agg({'synchtime': [min, max]})

cleaning up names

In [22]:
sum_Spire2.columns = ["_".join(x) for x in sum_Spire2.columns.ravel()]

creating columns for flavor shots, stripping flavor shots out of colnames

In [60]:
sum_Spire3 = Spire[['event', 'LMN', 'CHR', 'STW', 'VAN']]

In [61]:
sum_Spire3 = Spire.groupby('event').agg({'LMN': ['count'],'CHR':['count'],
                                         'STW':['count'], 'VAN':['count']})

In [62]:
sum_Spire3 = sum_Spire3[['LMN', 'CHR', 'STW', 'VAN']].astype('bool')

In [76]:
sum_Spire3['Flavor Shot'] = (sum_Spire3['LMN'] | sum_Spire3['CHR'] | sum_Spire3['STW'] | sum_Spire3['VAN'])

In [82]:
sum_Spire3 = sum_Spire3[['Flavor Shot', 'LMN', 'CHR', 'STW', 'VAN']]

In [49]:
# sum_Spire3

cleaning up colnames

In [63]:
sum_Spire3.columns = sum_Spire3.columns.droplevel(1)

joining df 1 and 3

In [83]:
sum_spire = sum_Spire.join(sum_Spire3).fillna(0)

In [1]:
# sum_spire.describe()

rejoining the two spire dataframes

In [24]:
Event_Spire = sum_spire.join(sum_Spire2)

In [4]:
# Event_Spire

In [26]:
Event_Spire.to_csv('Event_Spire.csv')

In [36]:
EyeQ['local_time_min'] = pd.to_datetime(EyeQ['local_time_min'])

In [6]:
# EyeQ.dtypes

In [5]:
# Event_Spire.dtypes

testing merge_asof for synching on time- didn't like this, used a better version we put into tian's algorith

In [54]:
data = pd.merge_asof(Event_Spire, EyeQ, left_on="synchtime_min", right_on="local_time_min", tolerance = pd.Timedelta('30s'),
                    direction = "nearest")

In [52]:
data.to_csv('data.csv')

In [7]:
# data